<h1> Deploying and predicting with model </h1>

This notebook illustrates:
<ol>
<li> Deploying model
<li> Predicting with model
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
  gsutil -m cp -R gs://cloud-training-demos/babyweight/trained_model gs://${BUCKET}/babyweight
fi

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [3]:
%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/

gs://cloud-training-demos-ml/babyweight/trained_model/export/exporter/
gs://cloud-training-demos-ml/babyweight/trained_model/export/exporter/1516730678/
gs://cloud-training-demos-ml/babyweight/trained_model/export/exporter/1516731289/
gs://cloud-training-demos-ml/babyweight/trained_model/export/exporter/1516731738/


In [ ]:
%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version 1.4

<h2> Use model to predict (online prediction) </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

In [4]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials)

request_data = {'instances':
  [
    {
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 39
    },
    {
      'is_male': 'False',
      'mother_age': 29.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 38
    },
    {
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Triplets(3)',
      'gestation_weeks': 39
    },
    {
      'is_male': 'Unknown',
      'mother_age': 29.0,
      'plurality': 'Multiple(2+)',
      'gestation_weeks': 38
    },
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'babyweight', 'ml_on_gcp')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)

[2018-01-23 21:50:04,882] {discovery.py:863} INFO - URL being requested: POST https://ml.googleapis.com/v1/projects/cloud-training-demos/models/babyweight/versions/ml_on_gcp:predict?alt=json
[2018-01-23 21:50:04,883] {client.py:614} INFO - Attempting refresh to obtain initial access_token
[2018-01-23 21:50:04,885] {client.py:903} INFO - Refreshing access_token
response={u'predictions': [{u'predictions': [7.659826755523682]}, {u'predictions': [7.222918510437012]}, {u'predictions': [6.318112373352051]}, {u'predictions': [6.186887264251709]}]}


The predictions for the four instances were: 7.66, 7.22, 6.32 and 6.19 pounds respectively when I ran it (your results might be different).

<h2> Use model to predict (batch prediction) </h2>
<p>
Batch prediction is commonly used when you thousands to millions of predictions.
Create a file withe one instance per line and submit using gcloud.
<p>
TODO: Note that the results are not sorted; you will have to use a key to match inputs to predictions and wrap the estimator using ```tf.contrib.estimator.forward_features```. This has not been done.

In [12]:
%writefile inputs.json
{"is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Writing inputs.json


In [ ]:
%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ml-engine jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=babyweight --version=soln

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License